In [1]:
# https://github.com/marqo-ai/marqo
# https://marqo.pages.dev/End-to-End%20Examples/streamlit_demo/

In [2]:
# This is for the marqo docker container to read files from local os. For more info on this please visit this link.
# https://github.com/marqo-ai/marqo/issues/35
# python3 -m http.server 8222

In [3]:
# sudo chmod 666 /var/run/docker.sock
# sudo docker rm -f marqo;docker run --name marqo -it --privileged -p 8882:8882 --add-host host.docker.internal:host-gateway marqoai/marqo:0.0.3

In [4]:
%%capture
!pip install wandb
!pip install marqo

In [5]:
import wandb
wandb.login()

wandb: Currently logged in as: a-sh0ts. Use `wandb login --relogin` to force relogin


True

In [6]:
project_name = "krea-open-prompts"
run_name = "insert-open-prompts-sd-marqo"
tags = ["download", "stable_diffusion", "insert", "marqo"]
_config = {
    # option of "prompts" or "sample_prompts"
    "dataset": "sample_prompts"
}

In [7]:
run = wandb.init(project=project_name, name=run_name, tags=tags, config=_config)

In [8]:
config = run.config
dataset_name = config['dataset']

In [9]:
dataset_path = run.use_artifact(f'{dataset_name}:latest', type='processed_data').download()

wandb: Downloading large artifact sample_prompts:latest, 618.73MB. 997 files... Done. 0:0:0.2


In [10]:
from pathlib import Path

In [11]:
df_path = Path(dataset_path, f"{dataset_name}.csv")

In [12]:
import pandas as pd
import modin.pandas as mpd
# from modin.config import ProgressBar
# ProgressBar.enable()

In [13]:
df = pd.read_csv(df_path)

In [14]:
#TODO: better way to prepend artifact path
df["local_image_location"] = str(dataset_path) + "/" + df["local_image_location"]

In [15]:
#Additional prepend for marqo
df["local_image_location"] = "http://host.docker.internal:8222/" + df["local_image_location"]

In [16]:
df["local_image_location"].iloc[0]

'http://host.docker.internal:8222/./artifacts/sample_prompts:v2/sample_prompts/A_portrait_photo_of_a_kangaroo_wearing_an_orange_hoodie_and_blue_sunglasses_standing_on_the_grass_in_front_of_the_Sydney_Opera_House_holding_a_sign_-C_15.0_-n_9_-i_-S_556046175_ts-1660001285_idx-4.png'

In [17]:
# import ray
# ray.init(ignore_reinit_error=True)

In [18]:
#BUG: Using tradiitonla pandas to flatten the dataframe before distributing in modin

In [19]:
data = df.to_dict('records')

In [20]:
from marqo import Client
from marqo.errors import MarqoApiError

In [31]:
marqo_settings = {
    "index_defaults": {
        "treat_urls_and_pointers_as_images": True,
        "image_preprocessing": {
            "patch_method": "frcnn"
        },
        "model":"ViT-B/16",
        "normalize_embeddings":True,
    },
}

In [32]:
mq = Client()

In [33]:
index_name = "stable-diffusion-open-prompts"

In [34]:
try:
    mq.create_index(index_name, settings_dict=marqo_settings)
    print("Risen from nothing, it wishes to learn")
except:
    mq.index(index_name).delete()
    mq.create_index(index_name, settings_dict=marqo_settings)
    print("It has been killed and brought back anew sire. Thirsty for knowledge")

It has been killed and brought back anew sire. Thirsty for knowledge


In [36]:
from tqdm import tqdm
from multiprocessing import cpu_count

In [37]:
device = 'cpu' #TODO: Add lightweight logic to see which gpu devices are available
NUM_CPUS = cpu_count()

In [39]:
# here we use parallel indexing to speed up the task
responses = mq.index(index_name).add_documents(data, device=device, processes=NUM_CPUS, batch_size=50)
# for data_doc in tqdm(data):
#     try:
#         responses = mq.index(index_name).add_documents([data_doc], device=device)
#         # print(f"<SUCCESS>\nAdded prompt:\n{data_doc['prompt']}\nURI: {data_doc['local_image_location']}\n")
#     except:
#         print(f"<FAILURE>\nSkipping prompt:\n{data_doc['prompt']}\nURI: {data_doc['local_image_location']}\n")

In [27]:
#TODO: Split into two notebooks
# First is half of this -> prep data for insertion
# That includes the transforms done here
# instead of ignoring the problematic opened images, let us pre-download all the images and place it alongside tha appropriate dataset
#     and have the image location be the name of the file where it will prepended with the artifact location after downlaoded
# If it is rate limit add some logic to check the files in an artifact against the table of downloaded files already
# and download that which was not downloaded and relog with that which was alrewady logged?
# Second will be the insertion which will simply pull the appropriate data down and log